In [5]:
import numpy as np
import scipy
from scipy import stats
from sklearn.ensemble import RandomForestClassifier

trainX = np.loadtxt('X_train.csv', delimiter=',', skiprows=1)
traint = np.loadtxt('y_train.csv', delimiter=',', skiprows=1)
traint = traint[:,1][:,None]-1
testX = np.loadtxt('X_test.csv', delimiter=',', skiprows=1)

def get_sk_model(trainX, traint):
    clf = RandomForestClassifier()
    model = clf.fit(trainX, traint.ravel())
    return model

def filter_by_variance(trainX, keep_count=10):
    sample_count = trainX.shape[0]
    feature_count = trainX.shape[1]
    variances = np.zeros(feature_count)
    
    # Find variance of each feature
    for i in range(feature_count):
        feature = trainX.T[i]
        variance = np.var(feature)
        variances[i] = variance    
    
    features_by_variance = variances.argsort()[:keep_count]

    newX = np.zeros((sample_count, keep_count))
    for i in range(keep_count):
        newX[:,i] = trainX[:,features_by_variance[i]]

    return newX, features_by_variance

def save_predictions(predictions, filename="predictions_forest.csv"):
    N = predictions.shape[0]
    output = np.ones((N, 2))
    output[:,0] = range(N)
    output[:,1] = predictions.ravel()
    np.savetxt(filename, output, fmt='%d', delimiter=",", header="Id,EpiOrStroma")
    print("Predictions saved")

# 22 is chosen as a heuristic
trainX, top_feature_indices = filter_by_variance(trainX, 22)
    
model = get_sk_model(trainX, traint)

# Select only top features if filtered
testX = testX[:,top_feature_indices]
preds = model.predict(testX)

save_predictions(preds+1)

Predictions saved
